In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup, Comment
import re
import pandas as pd
import os
import numpy as np


In [59]:
driver = webdriver.Remote(command_executor='http://127.0.0.1:9515/wd/hub', desired_capabilities=DesiredCapabilities.CHROME)

In [62]:
driver.get("https://adespresso.com/ads-examples/")

In [63]:
driver.find_element_by_id('ads-search').send_keys('football')
driver.find_element_by_id('ads-search').send_keys(Keys.ENTER)

In [38]:
 #Selenium hands the page source to Beautiful Soup
soup=BeautifulSoup(driver.page_source, 'lxml')


In [39]:
links = soup.find_all('ul','sf-results-list')

In [99]:
df = pd.DataFrame(columns=['Links'])
for b in links:
    for link in b.find_all('a'):
        d =link['href']
        df = df.append({'Links':d,}, ignore_index=True)      
            
            

In [69]:
def scrap():
    df = pd.DataFrame(columns=['Topic', 'Likes', 'Comments', 'Shares'])
    
    driver.get("https://adespresso.com/ad/2190951/?_return_to=https%3A%2F%2Fadespresso.com%2Fads-examples%2F%3Fsearch%3Dfootball%26placement%3D0%26industry%3D0%26objective%3D0%26attribute%3D0")

    soup1 = BeautifulSoup(driver.page_source, 'lxml')
    
    topic = soup1.find('h2')

    newUrl = soup1.find('iframe', {'allow': 'encrypted-media'})['src']
    driver.get(newUrl)

    soup2=BeautifulSoup(driver.page_source, 'lxml')
    like = soup2.find('span', {'class':'embeddedLikeButton'}).get_text()
    print(like)

    comment = soup2.find('div', {'title':'Comment'}).get_text()
    print(comment)

    share = soup2.find('div', {'title':'Share'}).get_text()
    print(share)
    
   
